## Step 1: Loopnest Scheduling

Run Timeloop-Topk using either the baseline model or the effective bandwidth model (considering the cryptographic engine)

In [1]:
%load_ext autoreload
%autoreload 2

import os
import yaml
import shutil
from pathlib import Path

from utils import generate_arch_files, xml2mapping 

### Define the architecture

First, define an architecture design. The code below generates/detects a new architecture configuration based on the template design at `designs/{design_name}/template`.

In [2]:
configuration_dict = {}

# template design (with constraints and memory hierarchy representing "dataflow")
configuration_dict['TEMPLATE_DESIGN'] = 'eyeriss_like'

# number of bits used for I/O/W; we assume integer
configuration_dict['WORDBITS'] = 16

# DRAM bandwidth setting: words / cycle (not bits / cycle)
configuration_dict['DRAM_READ_BANDWIDTH'] = 32
configuration_dict['DRAM_WRITE_BANDWIDTH'] = 32

# SRAM setting
# - do we have a single shared glb or multiple glbs for each datatype? 
# - for each glb (if shared, just one), define depth/width/#banks and bandwidths
configuration_dict['SRAM_SHARED'] = True
configuration_dict['SRAM_DEPTH'] = [2 ** 13]
configuration_dict['SRAM_WIDTH'] = [2 ** 7]
configuration_dict['SRAM_BANKS'] = [32]                     # SRAM width and SRAM banks define the maximum possible bandwidth
configuration_dict['SRAM_READ_BANDWIDTH'] = [32]
configuration_dict['SRAM_WRITE_BANDWIDTH'] = [32]

# PE array setting
# - shape of PE array X x Y
# - whether a PE has a shared scratchpad or separate scratchpads for each datatype
configuration_dict['PE_X'] = 14
configuration_dict['PE_Y'] = 12
configuration_dict['PE_SPAD_SHARED'] = False
configuration_dict['PE_SPAD_DEPTH'] = [192, 12, 16]         # Weight, IFmap, OFmap
configuration_dict['PE_SPAD_WIDTH'] = [16, 16, 16]

# Cryptographic engine setting
# - type of cryptographic engine + dram (LPDDR4 + AES-GCM)
# - cycle for AES-GCM 
# - whether the cryptographic engines are shared among all datatypes or assigned to each datatype
configuration_dict['CRYPT_ENGINE_TYPE'] = 'effective_lpddr4_aesgcm'
configuration_dict['CRYPT_ENGINE_CYCLE_PER_BLOCK'] = 1            # avg. cycle/128bit

configuration_dict['CRYPT_ENGINE_SHARED'] = False
configuration_dict['CRYPT_ENGINE_COUNT'] = [1, 1, 1]

configuration_dict['EFFECTIVE_CONSERVATIVE'] = True

# Create directory for this configuration if it doesn't exist already
# iterate through design folders to check if any pre-exisiting folder
design_dir = 'designs/{}'.format(configuration_dict['TEMPLATE_DESIGN'])
arch_dir = None
total_vers = 0
for path in os.listdir(design_dir):
    if path != 'template' and os.path.isdir(os.path.join(design_dir, path)):
        try:
            with open(os.path.join(design_dir, path, 'config.yaml'), 'r') as f:
                config_file = yaml.safe_load(f)
            total_vers += 1
            if config_file == configuration_dict:
                arch_dir = path
                print("Pre-existing folder found. Setting the arch_dir to {}".format(arch_dir))
                break
        except:
            print("No config.yaml file in the directory {}".format(str(os.path.join(design_dir, path))))
            
if arch_dir is None:
    arch_dir = 'ver{}'.format(total_vers)
    shutil.copytree(os.path.join(design_dir, 'template'), os.path.join(design_dir, arch_dir))
    with open(os.path.join(design_dir, arch_dir, 'config.yaml'), 'w') as f:
        _ = yaml.dump(configuration_dict, f)
    
    # create baseline and effective files
    generate_arch_files(os.path.join(design_dir, arch_dir, 'arch'), configuration_dict)
    
    # create scheduling / evaluation folder
    os.mkdir(os.path.join(design_dir, arch_dir, 'scheduling'))
    os.mkdir(os.path.join(design_dir, arch_dir, 'evaluation'))
    
    # create folders for baseline scheduling / evaluation
    os.mkdir(os.path.join(design_dir, arch_dir, 'baseline_scheduling'))
    os.mkdir(os.path.join(design_dir, arch_dir, 'baseline_evaluation'))

No config.yaml file in the directory designs/eyeriss_like/.ipynb_checkpoints
Pre-existing folder found. Setting the arch_dir to ver0


..else if you know which folder you want to use, specify here instead of running the above cell

In [ ]:
design_dir = 'designs/{}'.format('eyeriss_like') # define your design name here

arch_ver = 0
arch_dir = 'ver{}'.format(arch_ver)              # sub directory under designs/{name}/{arch_dir}
with open(os.path.join(design_dir, arch_dir, 'config.yaml'), 'r') as f:
    configuration_dict = yaml.safe_load(f)
print("Setting the architecture directory to: {}".format(os.path.join(design_dir, arch_dir)))
print("Printing configuration:")
for key, value in configuration_dict.items():
    print("{}: {}".format(key, value))

### Define the DNN workload

Define a DNN workload in PyTorch, and convert it into a Timeloop workload.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as model_zoo

import pytorch2timeloop as pytorch2timeloop

# Note: this version only supports nn.Conv2d (both normal convs and depthwise/pointwise convs) and nn.Linear

# AlexNet
model_name = 'alexnet'
net = model_zoo.alexnet(pretrained=False)

# ResNet18
# model_name = 'resnet18'
# net = model_zoo.resnet18(pretrained=False)

# MobilenetV2
# model_name = 'mobilenet_v2'
# net = model_zoo.mobilenet_v2(pretrained=False)

# Input / Batch info
input_size = (3, 224, 224)
batch_size = 1

print(net)

# Convert to timeloop workloads; stored in workloads/{model_name}_batch{batch_size}
top_dir = 'workloads'
sub_dir = '{}_batch{}'.format(model_name, batch_size)
exception_module_names = []

overwrite = False
if not os.path.exists(os.path.join(top_dir, sub_dir)) or overwrite:
    pytorch2timeloop.convert_model(
            net,
            input_size,
            batch_size,
            sub_dir,
            top_dir,
            True,
            exception_module_names
        )

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [4]:
# Check duplicate layers (layer information is identical)
# For a per-layer loopnest scheduling, only unique layers have to be searched

base_dir = Path(os.getcwd())
timeloop_dir = 'designs/{}/{}'.format(configuration_dict['TEMPLATE_DESIGN'], arch_dir)

n_layers = 0
layer_dict = {}
layer_duplicate_info = {}
unique_layers = []
for module in net.modules():
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        n_layers += 1
        if n_layers not in layer_dict.keys():
            workload_path = os.path.join(base_dir, top_dir, sub_dir, '{}_layer{}.yaml'.format(sub_dir, n_layers))
            with open(workload_path, 'r') as f:
                workload_info = yaml.safe_load(f)
            layer_dict[n_layers] = workload_info
        
        # identify the earliest duplicate layer
        for key in range(1, n_layers):
            if layer_dict[key] == layer_dict[n_layers]:
                layer_duplicate_info[n_layers] = key
                break
        if n_layers not in layer_duplicate_info:
            unique_layers.append(n_layers)
            
print(layer_duplicate_info)
print(unique_layers)

{}
[1, 2, 3, 4, 5, 6, 7, 8]


### Define the top-k parameter for Timeloop

Prepare the mapper.yaml for top-k search

In [5]:
topk = 6
mapper_file_path = os.path.join(base_dir, timeloop_dir, 'mapper/mapper.yaml')
with open(mapper_file_path, 'r') as f:
    mapper_config = yaml.safe_load(f)
mapper_config['mapper']['topk'] = topk
with open(mapper_file_path, 'w') as f:
    _ = yaml.dump(mapper_config, f)

### Run Timeloop for the *baseline* model (w/o considering cryptographic engines)

Run timeloop for each unique layer, and convert the output to a mapping file in yaml format. Then, evaluate using the top-1 loopnest schedule for each unique layer.

Running this cell can take some time depending on your model and timeloop setting..

In [ ]:
def get_cmd(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir):
    cwd = f"{base_dir/timeloop_dir/'baseline_scheduling'/sub_dir/f'layer{layer_id}'}"
    if 'M' in workload_info['problem']['instance']:
        constraint_pth = base_dir/timeloop_dir/'constraints/*.yaml'
    else:
        # depthwise
        constraint_pth = base_dir/timeloop_dir/'constraints_dw/*.yaml'

    timeloopcmd = f"timeloop-mapper-topk " \
                  f"{base_dir/timeloop_dir/'arch/baseline.yaml'} " \
                  f"{base_dir/timeloop_dir/'arch/components/*.yaml'} " \
                  f"{base_dir/timeloop_dir/'mapper/mapper.yaml'} " \
                  f"{constraint_pth} " \
                  f"{base_dir/top_dir/sub_dir/sub_dir}_layer{layer_id}.yaml "
    return [cwd, timeloopcmd]

cwd_list = []
cmd_list = []

for layer_id in unique_layers:
    workload_path = os.path.join(base_dir, top_dir, sub_dir, '{}_layer{}.yaml'.format(sub_dir, layer_id))
    with open(workload_path, 'r') as f:
        workload_info = yaml.safe_load(f)
    [cwd, cmd] = get_cmd(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir)
    cwd_list.append(cwd)
    cmd_list.append(cmd)
    
if not os.path.exists(os.path.join(base_dir, timeloop_dir, 'baseline_scheduling', sub_dir)):
    os.mkdir(os.path.join(base_dir, timeloop_dir, 'baseline_scheduling', sub_dir))
for cwd, cmd in zip(cwd_list, cmd_list):
    print("Executing cmd: {}".format(cmd))
    try:
        os.chdir(cwd)
    except:
        os.mkdir(cwd)
        os.chdir(cwd)
    os.system(cmd)
os.chdir(base_dir)

Convert to mapping (.yaml) files

In [ ]:
def convert_to_mapping(base_dir, timeloop_dir, top_dir, sub_dir, layer_idx, topk_idx):
    xml_file = os.path.join(base_dir, timeloop_dir, 'baseline_scheduling', sub_dir, "layer{}".format(layer_idx), \
                            "timeloop-mapper-topk{}.map+stats.xml".format(topk_idx))
    workload_file = os.path.join(base_dir, top_dir, sub_dir, "{}_layer{}.yaml".format(sub_dir, layer_idx))
    with open(workload_file, 'r') as f:
        workload_info = yaml.safe_load(f)
    if 'M' in workload_info['problem']['instance']:
        dw = False
    else:
        dw = True
    arch_constraint_file = os.path.join(base_dir, timeloop_dir, 'constraints_dw' if dw else 'constraints' , \
                                        'eyeriss_like_arch_constraints.yaml' if (configuration_dict['TEMPLATE_DESIGN'] == 'eyeriss_like' or \
                                                                                 configuration_dict['TEMPLATE_DESIGN'] == 'eyeriss_like_hbm2') \
                                        else 'simple_output_stationary_arch_constraints.yaml' if configuration_dict['TEMPLATE_DESIGN'] == 'output_stationary' \
                                        else 'simple_weight_stationary_arch_constraints.yaml')
    mapping = xml2mapping(xml_file, workload_file, arch_constraint_file, dw)
    with open(os.path.join(base_dir, timeloop_dir, 'baseline_scheduling',sub_dir, "layer{}".format(layer_idx), \
                           "mapping{}.yaml".format(topk_idx)), 'w') as f:
        _ = yaml.dump({'mapping': mapping}, f)
        
for layer_idx in unique_layers:
    for k in range(1, topk + 1):
        convert_to_mapping(base_dir, timeloop_dir, top_dir, sub_dir, layer_idx, k)

Evaluate the top-1 loopnest schedule

In [ ]:
def get_cmd_model(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir):
    cwd = f"{base_dir/timeloop_dir/'baseline_evaluation'/sub_dir/f'layer{layer_id}'}"
    if 'M' in workload_info['problem']['instance']:
        constraint_pth = base_dir/timeloop_dir/'constraints/*.yaml'
    else:
        # depthwise
        constraint_pth = base_dir/timeloop_dir/'constraints_dw/*.yaml'

    timeloopcmd = f"timeloop-model " \
                  f"{base_dir/timeloop_dir/'arch/baseline.yaml'} " \
                  f"{base_dir/timeloop_dir/'arch/components/*.yaml'} " \
                  f"{base_dir/timeloop_dir/'baseline_scheduling'/sub_dir/f'layer{layer_id}/mapping1.yaml'} " \
                  f"{base_dir/top_dir/sub_dir/sub_dir}_layer{layer_id}.yaml "
    return [cwd, timeloopcmd]

cwd_list = []
cmd_list = []
for layer_id in unique_layers:
    workload_path = os.path.join(base_dir, top_dir, sub_dir, '{}_layer{}.yaml'.format(sub_dir, layer_id))
    with open(workload_path, 'r') as f:
        workload_info = yaml.safe_load(f)
    [cwd, cmd] = get_cmd_model(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir)
    cwd_list.append(cwd)
    cmd_list.append(cmd)
    
if not os.path.exists(os.path.join(base_dir, timeloop_dir, 'baseline_evaluation', sub_dir)):
    os.mkdir(os.path.join(base_dir, timeloop_dir, 'baseline_evaluation', sub_dir))
for cwd, cmd in zip(cwd_list, cmd_list):
    print("Executing cmd: {}".format(cmd))
    try:
        os.chdir(cwd)
    except:
        os.mkdir(cwd)
        os.chdir(cwd)
    os.system(cmd)
os.chdir(base_dir)

### Run Timeloop for the *effective* model (considering cryptographic engines)

In [6]:
import time

def get_cmd(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir):
    cwd = f"{base_dir/timeloop_dir/'scheduling'/sub_dir/f'layer{layer_id}'}"
    if 'M' in workload_info['problem']['instance']:
        constraint_pth = base_dir/timeloop_dir/'constraints/*.yaml'
    else:
        # depthwise
        constraint_pth = base_dir/timeloop_dir/'constraints_dw/*.yaml'

    timeloopcmd = f"timeloop-mapper-topk " \
                  f"{base_dir/timeloop_dir/'arch/effective.yaml'} " \
                  f"{base_dir/timeloop_dir/'arch/components/*.yaml'} " \
                  f"{base_dir/timeloop_dir/'mapper/mapper.yaml'} " \
                  f"{constraint_pth} " \
                  f"{base_dir/top_dir/sub_dir/sub_dir}_layer{layer_id}.yaml "
    return [cwd, timeloopcmd]

cwd_list = []
cmd_list = []

for layer_id in unique_layers:
    workload_path = os.path.join(base_dir, top_dir, sub_dir, '{}_layer{}.yaml'.format(sub_dir, layer_id))
    with open(workload_path, 'r') as f:
        workload_info = yaml.safe_load(f)
    [cwd, cmd] = get_cmd(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir)
    cwd_list.append(cwd)
    cmd_list.append(cmd)
    
if not os.path.exists(os.path.join(base_dir, timeloop_dir, 'scheduling', sub_dir)):
    os.mkdir(os.path.join(base_dir, timeloop_dir, 'scheduling', sub_dir))
    
start_time = time.time()
for cwd, cmd in zip(cwd_list, cmd_list):
    print("Executing cmd: {}".format(cmd))
    try:
        os.chdir(cwd)
    except:
        os.mkdir(cwd)
        os.chdir(cwd)
    os.system(cmd)
os.chdir(base_dir)

# Time this cell
print("Execution time: {}s".format(time.time() - start_time))

Executing cmd: timeloop-mapper-topk /home/workspace/scheduling/designs/eyeriss_like/ver0/arch/effective.yaml /home/workspace/scheduling/designs/eyeriss_like/ver0/arch/components/*.yaml /home/workspace/scheduling/designs/eyeriss_like/ver0/mapper/mapper.yaml /home/workspace/scheduling/designs/eyeriss_like/ver0/constraints/*.yaml /home/workspace/scheduling/workloads/alexnet_batch1/alexnet_batch1_layer1.yaml 
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy /home/workspace/scheduling/designs/eyeriss_like/ver0/arch/effective.yaml /home/workspace/scheduling/designs/eyeriss_like/ver0/arch/components/smartbuffer_RF.yaml /home/workspace/scheduling/designs/eyeriss_like/ver0/arch/components/smartbuffer_SRAM.yaml /home/wo

[  3] Utilization = 0.33 | pJ/Compute =    8.194 | L5[WIO] Q11 C3 - L4[IO] M16 P55 Q5X - L3[] Q1 S11Y - L2[I] Q1 - L1[W] R11 - L0[O] M4 
[  7] Utilization = 0.26 | pJ/Compute =   14.134 | L5[WIO] Q55 M4 C3 - L4[IO] M4 P55 M4X - L3[] Q1 S11Y - L2[I] Q1 - L1[W] R11 - L0[O] Q1 
[  2] Utilization = 0.24 | pJ/Compute =   43.014 | L5[WIO] Q11 - L4[IO] M16 P55 C3 Q5X - L3[] Q1 S11Y - L2[I] Q1 - L1[W] R11 - L0[O] M4 
[  6] Utilization = 0.05 | pJ/Compute =  168.515 | L5[WIO] Q55 M4 - L4[IO] M4 P55 C3 M4X - L3[] Q1 S11Y - L2[I] Q1 - L1[W] R11 - L0[O] Q1 
[  6] Utilization = 0.05 | pJ/Compute =  173.150 | L5[WIO] Q55 M32 - L4[IO] P55 C3 - L3[] Q1 S11Y - L2[I] Q1 - L1[W] R11 - L0[O] M2 
[  5] STATEMENT: 200 invalid mappings (200 fanout, 0 capacity) found since the last valid mapping, terminating search.
[  7] Utilization = 0.07 | pJ/Compute =   44.657 | L5[WIO] Q55 M32 C3 - L4[IO] P55 - L3[] Q1 S11Y - L2[I] Q1 - L1[W] R11 - L0[O] M2 
[  1] STATEMENT: 200 invalid mappings (200 fanout, 0 capacity) 

[  7] Utilization = 0.72 | pJ/Compute =    7.759 | L5[WIO] Q5 M4 C3 - L4[IO] P55 Q11X - L3[] Q1 S11Y - L2[I] Q1 - L1[W] R11 - L0[O] M16 
[  6] Utilization = 0.51 | pJ/Compute =   22.451 | L5[WIO] Q5 M4 - L4[IO] P55 C3 Q11X - L3[] Q1 S11Y - L2[I] Q1 - L1[W] R11 - L0[O] M16 
[  2] Utilization = 0.51 | pJ/Compute =   23.654 | L5[WIO] Q5 M8 - L4[IO] P55 C3 Q11X - L3[] Q1 S11Y - L2[I] Q1 - L1[W] R11 - L0[O] M8 
[  7] Utilization = 0.72 | pJ/Compute =    8.246 | L5[WIO] Q5 M2 C3 - L4[IO] M8 P55 Q11X - L3[] Q1 S11Y - L2[I] Q1 - L1[W] R11 - L0[O] M4 
[  3] Utilization = 0.72 | pJ/Compute =    9.943 | L5[WIO] Q5 M8 C3 - L4[IO] P55 Q11X - L3[] Q1 S11Y - L2[I] Q1 - L1[W] R11 - L0[O] M8 
[  6] Utilization = 0.51 | pJ/Compute =   26.059 | L5[WIO] Q5 M2 - L4[IO] M8 P55 C3 Q11X - L3[] Q1 S11Y - L2[I] Q1 - L1[W] R11 - L0[O] M4 
[  2] Utilization = 0.51 | pJ/Compute =   23.654 | L5[WIO] Q5 M4 - L4[IO] M2 P55 C3 Q11X - L3[] Q1 S11Y - L2[I] Q1 - L1[W] R11 - L0[O] M8 
[  3] Utilization = 0.72 | pJ/Compute



Summary stats for best mapping found by mapper:
  Utilization = 0.72 | pJ/Compute =    6.459

Summary stats for best mapping found by mapper:
  Utilization = 0.72 | pJ/Compute =    6.892

Summary stats for best mapping found by mapper:
  Utilization = 0.72 | pJ/Compute =    6.910

Summary stats for best mapping found by mapper:
  Utilization = 0.72 | pJ/Compute =    7.344

Summary stats for best mapping found by mapper:
  Utilization = 0.72 | pJ/Compute =    7.759

Summary stats for best mapping found by mapper:
  Utilization = 0.72 | pJ/Compute =    7.813
Executing cmd: timeloop-mapper-topk /home/workspace/scheduling/designs/eyeriss_like/ver0/arch/effective.yaml /home/workspace/scheduling/designs/eyeriss_like/ver0/arch/components/*.yaml /home/workspace/scheduling/designs/eyeriss_like/ver0/mapper/mapper.yaml /home/workspace/scheduling/designs/eyeriss_like/ver0/constraints/*.yaml /home/workspace/scheduling/workloads/alexnet_batch1/alexnet_batch1_layer2.yaml 
  _______                _

[  4] Utilization = 0.06 | pJ/Compute =   13.906 | L5[WIO] Q27 M8 C64 - L4[IO] P27 - L3[] Q1 M2Y S5Y - L2[I] Q1 - L1[W] R5 - L0[O] M12 
[  3] Utilization = 0.18 | pJ/Compute =   11.858 | L5[WIO] Q27 C32 - L4[IO] M16 P27 M3X - L3[] Q1 C2Y S5Y - L2[I] Q1 - L1[W] R5 - L0[O] M4 
[  2] Utilization = 0.09 | pJ/Compute =   11.834 | L5[WIO] Q27 C32 - L4[IO] M16 P27 M3X - L3[] Q1 S5Y - L2[I] Q1 - L1[W] R5 C2 - L0[O] M4 
[  4] Utilization = 0.18 | pJ/Compute =   13.243 | L5[WIO] Q27 M2 C64 - L4[IO] M16 P27 M6X - L3[] Q1 S5Y - L2[I] Q1 - L1[W] R5 - L0[O] Q1 
[  6] Utilization = 0.42 | pJ/Compute =   28.628 | L5[WIO] Q3 M4 C2 - L4[IO] M12 P27 C16 Q9X - L3[] Q1 C2Y S5Y - L2[I] Q1 - L1[W] R5 - L0[O] M4 
[  5] Utilization = 0.27 | pJ/Compute =   28.627 | L5[WIO] Q3 M4 C2 - L4[IO] M12 P27 C16 Q9X - L3[] Q1 S5Y - L2[I] Q1 - L1[W] R5 C2 - L0[O] M4 
[  7] Utilization = 0.05 | pJ/Compute =  165.985 | L5[WIO] Q27 M2 C4 - L4[IO] M16 P27 C4 M6X - L3[] Q1 C2Y S5Y - L2[I] Q1 - L1[W] R5 C2 - L0[O] Q1 
[  2] Uti

...

[  2] Utilization = 0.04 | pJ/Compute =  186.540 | L5[WIO] M125 C8 - L4[IO] C256 M2X - L3[] Q1 M4Y C2Y - L2[I] Q1 - L1[W] Q1 - L0[O] Q1 
[  6] Utilization = 0.05 | pJ/Compute =  163.475 | L5[WIO] M5 C8 - L4[IO] M5 C128 M8X - L3[] Q1 - L2[I] Q1 - L1[W] C4 - L0[O] M5 
[  4] Utilization = 0.04 | pJ/Compute =  186.540 | L5[WIO] M125 C8 - L4[IO] C32 M2X - L3[] Q1 M4Y C2Y - L2[I] Q1 - L1[W] C8 - L0[O] Q1 
[  7] Utilization = 0.01 | pJ/Compute =  166.889 | L5[WIO] M10 C128 - L4[IO] M10 C16 - L3[] Q1 - L2[I] Q1 - L1[W] C2 - L0[O] M10 
[  7] Utilization = 0.05 | pJ/Compute =  164.204 | L5[WIO] M50 - L4[IO] M2 C256 - L3[] Q1 M5Y C2Y - L2[I] Q1 - L1[W] C8 - L0[O] M2 
[  1] Utilization = 0.04 | pJ/Compute =  186.540 | L5[WIO] M125 C8 - L4[IO] C256 M2X - L3[] Q1 M4Y - L2[I] Q1 - L1[W] C2 - L0[O] Q1 
[  3] Utilization = 0.05 | pJ/Compute =  163.476 | L5[WIO] M5 C8 - L4[IO] M5 C32 M8X - L3[] Q1 C4Y - L2[I] Q1 - L1[W] C4 - L0[O] M5 
[  5] Utilization = 0.05 | pJ/Compute =  164.324 | L5[WIO] M4 C128 - 

[  2] Utilization = 0.05 | pJ/Compute =  168.366 | L5[WIO] M20 C256 - L4[IO] M5 M2X - L3[] Q1 C4Y - L2[I] Q1 - L1[W] C4 - L0[O] M5 
[  3] Utilization = 0.05 | pJ/Compute =  168.428 | L5[WIO] M20 C256 - L4[IO] M5 M2X - L3[] Q1 C8Y - L2[I] Q1 - L1[W] C2 - L0[O] M5 
[  7] Utilization = 0.05 | pJ/Compute =  164.322 | L5[WIO] Q1 - L4[IO] M100 C4096 M5X - L3[] Q1 M2Y - L2[I] Q1 - L1[W] Q1 - L0[O] Q1 
[  2] Utilization = 0.05 | pJ/Compute =  163.970 | L5[WIO] M2 C64 - L4[IO] M25 C32 M5X - L3[] Q1 C2Y - L2[I] Q1 - L1[W] Q1 - L0[O] M4 
[  3] Utilization = 0.03 | pJ/Compute =  163.966 | L5[WIO] M2 C64 - L4[IO] M25 C64 M5X - L3[] Q1 - L2[I] Q1 - L1[W] Q1 - L0[O] M4 
[  4] Utilization = 0.05 | pJ/Compute =  162.466 | L5[WIO] C8 - L4[IO] M10 C32 M5X - L3[] Q1 M2Y C4Y - L2[I] Q1 - L1[W] C4 - L0[O] M10 
[  5] Utilization = 0.03 | pJ/Compute =  172.824 | L5[WIO] M50 C4 - L4[IO] M2 C512 M5X - L3[] Q1 - L2[I] Q1 - L1[W] C2 - L0[O] M2 
[  6] Utilization = 0.05 | pJ/Compute =  172.824 | L5[WIO] M50 C4 - L

[  3] Utilization = 0.05 | pJ/Compute =  165.513 | L5[WIO] M5 C64 - L4[IO] M20 C2 M5X - L3[] Q1 C4Y - L2[I] Q1 - L1[W] C8 - L0[O] M2 
[  5] Utilization = 0.05 | pJ/Compute =  164.052 | L5[WIO] M2 C64 - L4[IO] M25 C4 M10X - L3[] Q1 C8Y - L2[I] Q1 - L1[W] C2 - L0[O] M2 
[  4] Utilization = 0.05 | pJ/Compute =  164.036 | L5[WIO] M2 C64 - L4[IO] M25 C4 M10X - L3[] Q1 C4Y - L2[I] Q1 - L1[W] C4 - L0[O] M2 
[  3] Utilization = 0.05 | pJ/Compute =  164.029 | L5[WIO] M2 C64 - L4[IO] M25 C4 M10X - L3[] Q1 C2Y - L2[I] Q1 - L1[W] C8 - L0[O] M2 
[  7] Utilization = 0.05 | pJ/Compute =  164.052 | L5[WIO] M2 C64 - L4[IO] M25 C2 M10X - L3[] Q1 C8Y - L2[I] Q1 - L1[W] C4 - L0[O] M2 
[  3] Utilization = 0.05 | pJ/Compute =  164.665 | L5[WIO] C256 - L4[IO] M25 M8X - L3[] Q1 C4Y - L2[I] Q1 - L1[W] C4 - L0[O] M5 
[  2] Utilization = 0.05 | pJ/Compute =  162.393 | L5[WIO] C8 - L4[IO] M5 C512 M8X - L3[] Q1 M5Y - L2[I] Q1 - L1[W] Q1 - L0[O] M5 
[  0] Utilization = 0.05 | pJ/Compute =  162.852 | L5[WIO] C16 - L

[  3] Utilization = 0.05 | pJ/Compute =  162.405 | L5[WIO] C8 - L4[IO] M5 C128 M10X - L3[] Q1 M5Y - L2[I] Q1 - L1[W] C4 - L0[O] M4 
[  7] Utilization = 0.05 | pJ/Compute =  162.406 | L5[WIO] C8 - L4[IO] M5 C64 M10X - L3[] Q1 M5Y C2Y - L2[I] Q1 - L1[W] C4 - L0[O] M4 
[  7] Utilization = 0.05 | pJ/Compute =  162.332 | L5[WIO] C4 - L4[IO] M5 C256 M10X - L3[] Q1 M2Y C4Y - L2[I] Q1 - L1[W] Q1 - L0[O] M10 
[  1] Utilization = 0.05 | pJ/Compute =  162.303 | L5[WIO] M5 - L4[IO] C128 M10X - L3[] Q1 M2Y C4Y - L2[I] Q1 - L1[W] C8 - L0[O] M10 
[  7] Utilization = 0.05 | pJ/Compute =  162.384 | L5[WIO] M2 - L4[IO] M4 C1024 M5X - L3[] Q1 M5Y C2Y - L2[I] Q1 - L1[W] C2 - L0[O] M5 
[  4] Utilization = 0.05 | pJ/Compute =  162.313 | L5[WIO] C2 - L4[IO] M5 C256 M10X - L3[] Q1 M2Y - L2[I] Q1 - L1[W] C8 - L0[O] M10 
[  3] Utilization = 0.05 | pJ/Compute =  162.432 | L5[WIO] C2 - L4[IO] M10 C1024 M4X - L3[] Q1 M5Y - L2[I] Q1 - L1[W] C2 - L0[O] M5 
[  4] Utilization = 0.05 | pJ/Compute =  162.303 | L5[WIO] Q



Summary stats for best mapping found by mapper:
  Utilization = 0.05 | pJ/Compute =  162.376

Summary stats for best mapping found by mapper:
  Utilization = 0.05 | pJ/Compute =  162.399

Summary stats for best mapping found by mapper:
  Utilization = 0.05 | pJ/Compute =  162.406

Summary stats for best mapping found by mapper:
  Utilization = 0.05 | pJ/Compute =  162.332

Summary stats for best mapping found by mapper:
  Utilization = 0.05 | pJ/Compute =  162.384

Summary stats for best mapping found by mapper:
  Utilization = 0.05 | pJ/Compute =  162.336
Execution time: 128.90579557418823s


In [7]:
def convert_to_mapping(base_dir, timeloop_dir, top_dir, sub_dir, layer_idx, topk_idx):
    xml_file = os.path.join(base_dir, timeloop_dir, 'scheduling', sub_dir, "layer{}".format(layer_idx), \
                            "timeloop-mapper-topk{}.map+stats.xml".format(topk_idx))
    workload_file = os.path.join(base_dir, top_dir, sub_dir, "{}_layer{}.yaml".format(sub_dir, layer_idx))
    # print(workload_file)
    with open(workload_file, 'r') as f:
        workload_info = yaml.safe_load(f)
    if 'M' in workload_info['problem']['instance']:
        dw = False
    else:
        dw = True
    arch_constraint_file = os.path.join(base_dir, timeloop_dir, 'constraints_dw' if dw else 'constraints' , \
                                        'eyeriss_like_arch_constraints.yaml' if (configuration_dict['TEMPLATE_DESIGN'] == 'eyeriss_like' \
                                                                                 or configuration_dict['TEMPLATE_DESIGN'] == 'eyeriss_like_hbm2') \
                                        else 'simple_output_stationary_arch_constraints.yaml' if configuration_dict['TEMPLATE_DESIGN'] == 'output_stationary' \
                                        else 'simple_weight_stationary_arch_constraints.yaml')
    # print(layer_idx, dw)
    mapping = xml2mapping(xml_file, workload_file, arch_constraint_file, dw)
    with open(os.path.join(base_dir, timeloop_dir, 'scheduling',sub_dir, "layer{}".format(layer_idx), \
                           "mapping{}.yaml".format(topk_idx)), 'w') as f:
        _ = yaml.dump({'mapping': mapping}, f)
        
for layer_idx in unique_layers:
    for k in range(1, topk + 1):
        convert_to_mapping(base_dir, timeloop_dir, top_dir, sub_dir, layer_idx, k)

In [8]:
def get_cmd_model(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir):
    cwd = f"{base_dir/timeloop_dir/'evaluation'/sub_dir/f'layer{layer_id}'}"
    if 'M' in workload_info['problem']['instance']:
        constraint_pth = base_dir/timeloop_dir/'constraints/*.yaml'
    else:
        # depthwise
        constraint_pth = base_dir/timeloop_dir/'constraints_dw/*.yaml'

    timeloopcmd = f"timeloop-model " \
                  f"{base_dir/timeloop_dir/'arch/baseline.yaml'} " \
                  f"{base_dir/timeloop_dir/'arch/components/*.yaml'} " \
                  f"{base_dir/timeloop_dir/'scheduling'/sub_dir/f'layer{layer_id}/mapping1.yaml'} " \
                  f"{base_dir/top_dir/sub_dir/sub_dir}_layer{layer_id}.yaml "
    return [cwd, timeloopcmd]

cwd_list = []
cmd_list = []
for layer_id in unique_layers:
    workload_path = os.path.join(base_dir, top_dir, sub_dir, '{}_layer{}.yaml'.format(sub_dir, layer_id))
    with open(workload_path, 'r') as f:
        workload_info = yaml.safe_load(f)
    [cwd, cmd] = get_cmd_model(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir)
    cwd_list.append(cwd)
    cmd_list.append(cmd)
    
if not os.path.exists(os.path.join(base_dir, timeloop_dir, 'evaluation', sub_dir)):
    os.mkdir(os.path.join(base_dir, timeloop_dir, 'evaluation', sub_dir))
for cwd, cmd in zip(cwd_list, cmd_list):
    print("Executing cmd: {}".format(cmd))
    try:
        os.chdir(cwd)
    except:
        os.mkdir(cwd)
        os.chdir(cwd)
    os.system(cmd)
os.chdir(base_dir)

Executing cmd: timeloop-model /home/workspace/scheduling/designs/eyeriss_like/ver0/arch/baseline.yaml /home/workspace/scheduling/designs/eyeriss_like/ver0/arch/components/*.yaml /home/workspace/scheduling/designs/eyeriss_like/ver0/scheduling/alexnet_batch1/layer1/mapping1.yaml /home/workspace/scheduling/workloads/alexnet_batch1/alexnet_batch1_layer1.yaml 
execute:/usr/local/bin/accelergy /home/workspace/scheduling/designs/eyeriss_like/ver0/arch/baseline.yaml /home/workspace/scheduling/designs/eyeriss_like/ver0/arch/components/smartbuffer_RF.yaml /home/workspace/scheduling/designs/eyeriss_like/ver0/arch/components/smartbuffer_SRAM.yaml /home/workspace/scheduling/designs/eyeriss_like/ver0/scheduling/alexnet_batch1/layer1/mapping1.yaml /home/workspace/scheduling/workloads/alexnet_batch1/alexnet_batch1_layer1.yaml --oprefix timeloop-model. -o ./ > timeloop-model.accelergy.log 2>&1
Utilization = 0.72 | pJ/Compute =    6.267
Executing cmd: timeloop-model /home/workspace/scheduling/designs/ey